In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#Importing the necessary libraries
import os
import torch
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from train import train_model
from utils import test_latency_FPS
from utils import test_FLOPs_params

In [4]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

cityscapes_train_dataset = CityscapesCustom(cityscapes_dir, 'train', height=8, width=16)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', height=8, width=16)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=4, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=4, shuffle=False)

print(f'Train: {len(cityscapes_train_dataset)} images, divided into {len(cityscapes_train_dataloader)} batches of size {cityscapes_train_dataloader.batch_size}')
print(f'Test: {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [6]:
#Testing Classic semantic segmentation network

#Set up the model with the pretrained weights
DeepLabV2_model = get_deeplab_v2().to(device)

#Set up the loss function and the optimizer
DeepLabV2_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
DeepLabV2_optimizer = optim.Adam(DeepLabV2_model.parameters(), lr=1e-3)

#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Deeplab pretraining loading...


In [7]:
#Train the model
train_model(DeepLabV2_model, DeepLabV2_criterion, DeepLabV2_optimizer, cityscapes_train_dataloader, cityscapes_test_dataloader, device, n_epochs=10, lr_schedule=False)

#Test latency and FPS
#print(test_latency_FPS(DeepLabV2_model, height=4, width=8, device=device))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(DeepLabV2_model, height=4, width=8, device=device))

Epoch 1/10, IoU: 17.31% (153/884) [13.99s]
Epoch 2/10, IoU: 36.88% (326/884) [12.80s]
Epoch 3/10, IoU: 51.36% (454/884) [13.23s]
Epoch 4/10, IoU: 60.18% (532/884) [13.92s]
Epoch 5/10, IoU: 76.02% (672/884) [12.88s]
Epoch 6/10, IoU: 74.89% (662/884) [12.40s]
Epoch 7/10, IoU: 72.96% (645/884) [12.58s]
Epoch 8/10, IoU: 72.06% (637/884) [12.36s]
Epoch 9/10, IoU: 79.52% (703/884) [12.27s]
Epoch 10/10, IoU: 47.85% (423/884) [12.96s]
Best IoU: 79.52% at epoch 9


In [8]:
#Testing Real-Time semantic segmentation network

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [9]:
#Train the model
train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer, cityscapes_train_dataloader, cityscapes_test_dataloader, device, n_epochs=10, lr_schedule=True)

#Test latency and FPS
#print(test_latency_FPS(BiSeNet_model, height=4, width=8, device=device))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(BiSeNet_model, height=4, width=8, device=device))

Epoch 1/10, IoU: 46.83% (414/884) [7.87s]
Epoch 2/10, IoU: 43.55% (385/884) [7.63s]
Epoch 3/10, IoU: 42.19% (373/884) [7.80s]
Epoch 4/10, IoU: 46.27% (409/884) [8.04s]
Epoch 5/10, IoU: 47.29% (418/884) [8.48s]
Epoch 6/10, IoU: 43.55% (385/884) [10.79s]
Epoch 7/10, IoU: 46.61% (412/884) [10.89s]
Epoch 8/10, IoU: 47.29% (418/884) [9.89s]
Epoch 9/10, IoU: 45.25% (400/884) [8.82s]
Epoch 10/10, IoU: 43.44% (384/884) [8.19s]
Best IoU: 47.29% at epoch 5
